# Change Detection - Image Ratio Demo
This notebook detects change between satellite images taken at subsequent times over the same location.

The methods are calibrated on [Sentinel-2](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/overview) pre-post disaster imagery from the Caribbean and [Copernicus Emergency Mapping Service (EMS)](https://emergency.copernicus.eu/mapping/map-of-activations-rapid#zoom=3&lat=29.18235&lon=-70.57787&layers=BT00) ground truth damage assessment data. Section 4 allows you to evaluate predicted damages against reported damages for locations covered by EMS. The models were built using the [Descartes Labs](https://www.descarteslabs.com/) platform. If you wish to re-train the models or have more flexibility in your parameters, plots or methods, please visit the notebooks for each method at [thresholding.ipynb](./thresholding.ipynb) and [unet_classifier.ipynb](./unet_classifier.ipynb).
### Contents
- 1 - [Visualise Subsequent Imagery](#visualiseImagery)
- 2 - [Detect Change - Thresholding Method](#thresholding)
- 3 - [Detect Change - Trained Classifier Method](#classifier)
- 4 - [Evaluate Methods Against Ground Data](#evaluate)
____________
## Initialisation - Choose Location
Firstly we need to choose a location for which to detect change. You may either choose one of the locations with pre-assigned variables from the list displayed when running the first cell, or you can choose a new location by selecting 'New Location' which will then prompt you to provide variables of your own when running the second cell. Pre-set locations are the following:
- Training Set: Roseau, Dominica (pre-post hurricane Maria); Abricots, Haiti (pre-post hurricane Matthew)
- Test Set: Jeremie, Haiti (pre-post hurricane Matthew); Port Salut, Haiti (pre-post hurricane Matthew)
- Informal Settlements: Bidonville Killick Stenio Vincent, Port-au-Prince, Haiti (2018 to 2019); Parry Town, Ocho Rios, Jamaica (2018 to 2019)
- High Resolution: Hidalgo County, Texas, USA (2016 to 2019)

In [1]:
import demoFunctions as dfn # Import functions from demoFunction.py
location = dfn.chooseLocation() # Generate pre-defined location list
location # Display location dropdown

Dropdown(description='Location:', options=(('train - Roseau, Dominica', 0), ('train - Abricots, Haiti', 1), ('…

In [2]:
# Assign variables for chosen location
variables = dfn.assignVariables(location) 

FloatText(value=18.6486, description='Latitude (decimal):')

FloatText(value=-74.3058, description='Longitude (decimal):')

IntText(value=16, description='Map zoom:')

Text(value='2016-07-01', description='Image 1 start date:', placeholder='yyyy-mm-dd')

Text(value='2016-09-30', description='Image 1 end date:', placeholder='yyyy-mm-dd')

...

FloatText(value=0.01, description='Default threshold:')

FloatText(value=0.1, description='Default Cap')

Dropdown(description='Imagery bands:', options=(('red', ['red']), ('green', ['green']), ('blue', ['blue']), ('…

IntText(value=10, description='Satellite resolution:')

Text(value='', description='Path to mask if using one:', placeholder='mask.geojson')

In [3]:
# Submits input variables for new location (if not new location - no action)
variables = dfn.submitNewLocation(variables)

_____________
<a id='visualiseImagery'></a>
## 1 - Visualise Imagery
We'll start by displaying before and after images for the chosen location. Two pointers here:
- You'll need to click the magic markers below the map to scale the imagery band colours properly. You can also untick the 'After' box to toggle between pre and post disaster.
- If you have defined a new location and no imagery appears for your location, try increasing the cloud fraction, but check this does not lead to overly cloudy images which may affect detection performance. Alternatively try changing or widening the requested dates, it is not uncommon to span several months for a good image.

In [4]:
# Create map with before and after images for specified location
dfn.beforeAfterImages(variables)


`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart the kernel and refresh the webpage.


________
<a id='thresholding'></a>
## 2 - Detect Change - Ratio Thresholding
Next, let's take the logarithmic ratio of images for selected bands and display detected change superimposed on the image for time 2. Beneath the plot is a slider allowing you to vary the thresholds within which we are detecting change. If damage assessment data is available for the location this will be overlayed for a qualitative comparison.

> The optimal threshold interval for detecting building change has been determined as 0.01-0.1. However, due to differences in lighting between repeat images this may need adjsuted according to location. As the ratio is logarithmic, if the second image is considerably darker than the first, the appropriate thresholding may even be negative.

Detection interval equation for RGB (red, green and blue bands) where, for example, r1 denotes pixel value for red band at time 1:  $ threshold < \log\left(\frac{r2}{r1} \times \frac{b2}{b1} \times \frac{g2}{g1}\right) < cap  $

In [5]:
# Function to detect change through thresholding for location
plotVars = dfn.thresholding(variables)

`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jup…

FloatRangeSlider(value=(0.01, 0.1), description='Filter ratios', max=0.5, min=-0.5, step=0.01)

Run box below to display updated detection result


In [6]:
# Re-plot detected change according to slider values
dfn.plotChange(plotVars)

You may observe 2 main limitations with this method.
- Lack of small scale change detections due to 10x10m resolution of Sentinel Imagery. Try re-running the notebook selecting the high resolution location to see what is possible where 1x1m resolution is available.
- Sensitivity of thresholding to image lighting and changes in areas other than buildings, due to effect on ratio values. If threshold is unadjusted or mask is not applied to ocean this leads to false detections. To mitigate this limitation we trained a [classifier](#classifier) in the next section to recognise the evidence of damage buildings within the image ratio rather than relying on simple thresholding.

____________________
<a id='classifier'></a>
## 3 - Detect Change - Ratio Classifier
To avoid the drawbacks of thresholding we trained a Convolutional Neural Network (CNN) with a [U-Net](https://arxiv.org/abs/1505.04597) architecture to identify building change from the pixel ratios between before/after Sentinel-2 imagery. This model evaluates change per 'image tile' of which there will be many within your area of interest. Therefore, first let's draw a polygon over the desired area. Then, each corresponding tile will individually be fed in to the model for assessing change detection. Bare in mind the larger the area the longer it will take to run the model for that area.
> One could question the decision not to just increase tilesize. However not only does this method make the evaluation area more flexible, but also the model does not cater well for tile sizes larger than that for which it was trained due to the input layer structure.

In [16]:
m3, testPoly = dfn.drawPolygon(variables) # Get map upon which to draw polygon for assessment
m3 # Display map


`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart the kernel and refresh the webpage.


In [18]:
# Detect change for all tiles at least partly within polygon
dfn.classifyDamage(testPoly, variables, m3)

Longitude Columns:   0%|          | 0/5 [00:00<?, ?it/s]

Number of tiles requested: 15 . Approximately 120 seconds on 16GB RAM.

Job ID: 0a44131df9ee8b14bf6611be17eaca6f5cc7bb1b9b54bc42
[######] | Steps: 5/5 | Stage: SUCCEEDED                                      


Longitude Columns:  20%|██        | 1/5 [00:05<00:23,  5.99s/it]


Job ID: ac5ed63804b81f6f21823cf8832153e0b4796865f82c26be
[######] | Steps: 5/5 | Stage: SUCCEEDED                                      


Longitude Columns:  40%|████      | 2/5 [00:12<00:18,  6.16s/it]

...


Longitude Columns:  60%|██████    | 3/5 [00:17<00:12,  6.02s/it]


Job ID: e047cc145f09677a4e1790f7f3789a7b75ffa52c5977f003
[######] | Steps: 5/5 | Stage: SUCCEEDED                                      WARNING:tensorflow:11 out of the last 11 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7f7df4313a70> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.



Longitude Columns:  80%|████████  | 4/5 [00:23<00:06,  6.01s/it]


Job ID: e729771a174eddeccc2cd5433f9f11d2212dfc943ec48323
[######] | Steps: 5/5 | Stage: SUCCEEDED                                      WARNING:tensorflow:11 out of the last 11 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7f7df2a40170> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.



Latitude Rows: 100%|██████████| 3/3 [01:29<00:00, 29.68s/it]


_________________
<a id='evaluate'></a>
## 4 - Evaluate Methods Against Ground Data
We can now use Copernicus EMS post-disaster damage assessments to evaluate our models' performance at detecting building change. We will first plot a map evaluating thresholding accuracy, and then for classifier accuracy.

In [ ]:
## Evaluate against damages
if not deployed:
    # Load building damages and filter for within detection area
    dmg = gpd.read_file(dmgJsons)
    filtered = gpd.GeoDataFrame(crs={'init': 'epsg:4326'})

    tilePoly = gpd.GeoSeries(Polygon.from_bounds(min(allCtx[0::4]),min(allCtx[1::4]),max(allCtx[2::4]),max(allCtx[3::4])), crs={'init':ctx.bounds_crs}).to_crs(epsg=4326).geometry[0]
    for i in dmg.index: 
        if dmg.geometry[i].centroid.within(tilePoly):
            filtered = filtered.append(dmg.loc[i])

    print('Changed pixels:',len(allDet), '\nDamaged buildings:',len(filtered))

    # Initialise accuracy and recall vectors
    acc, rec = np.zeros([max(filtered.index)+1,1]), np.zeros([max(allDet.index)+1,1]) # Initialise accuracy, recall arrays

    # Loop through pixels to determine recall (if pixel corresponds to damaged building)
    for i in tqdm(allDet.index):
        # Loop through building to determine accuracy (damaged building has been detected)
        for j in filtered.index:
            if allDet.geometry[i].within(filtered.geometry[j]):
                rec[i,0], acc[j,0] = True, True

    # Calculate metrics from vector outputs
    a = sum(acc)/len(filtered)
    r = sum(rec)/len(allDet)
    f1 = 2*(a*r)/(a+r)
    print('Accuracy:',a[0],'\nRecall:',r[0],'\nF1 score:',f1[0])    

__________
<a id='evaluate'></a>
## 3 - Evaluate Detection Model


In [ ]:
# Convert mask into coordinate array
# Get vector of pixels which have changed coordinates
gtx, detection = wf.map.geocontext(), detections.mask(detections==0).mask(omitMask==1) if os.path.exists(maskPoly) else detections.mask(detections==0)
change = detection.compute(geoctx=gtx)

# Get latitude & longitude of each pixel in mask (whether true or false)
bounds = change.geocontext['bounds']
lats, longs = np.linspace(bounds[3],bounds[1],change.geocontext['arr_shape'][0]), np.linspace(bounds[0],bounds[2],change.geocontext['arr_shape'][1])

# Create matrix of coordinates for pixels with change detected
xm, ym = np.meshgrid(longs,lats)
xc, yc = xm*(1-change.ndarray.mask[0]), ym*(1-change.ndarray.mask[0])

# Get geodataframe for pixel points
df = pd.DataFrame(columns=['Latitude', 'Longitude'])
for i,j in tqdm(zip(np.nonzero(xc)[0], np.nonzero(xc)[1])):
    df = df.append({'Latitude': yc[i][j],'Longitude': xc[i][j]}, ignore_index=True)
    
det = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

In [ ]:
# Load in damage geojson from Copernicus EMS data
try: 
    settlements = gpd.read_file(dmgAssess).to_crs({'init': 'epsg:4326'})
    color_dict = {'Not Applicable':'green','Negligible to slight damage':'blue', 'Moderately Damaged':'yellow', 'Highly Damaged':'orange', 'Completely Destroyed':'red'}

    # Filter settlements to be within specified damage grade and location polygon
    damage = settlements[settlements.grading.isin(grades)]
    damage = damage[damage.within(Polygon(change.geocontext['geometry']['coordinates'][0]))]

    # Filter detections to area covered by damage inspection
    try: det = det.loc[(det.geometry.x < max(damage.geometry.x)) & (det.geometry.x > min(damage.geometry.x)) & (det.geometry.y < max(damage.geometry.y)) & (det.geometry.y > min(damage.geometry.y))]
    except: pass
except: print("No damage file set for this area. All further steps are for assessing detection against a damage assessment")

In [ ]:
# Plot detected vs damaged in matplotlib
# Plot damages
try: # For assessments with damage points
    plt.figure(figsize=(10,6))
    plt.scatter(damage.geometry.x,damage.geometry.y,color=[color_dict[i] for i in damage.grading],s=8,alpha=0.2,label='Building damage')
except:  # For assessments with damage polygons
    try: plt = damage.plot(figsize=(10,6),column='grading',alpha=1.,legend=True,cmap='RdYlGn')
    except: print('As previously mentioned => No damage file to plot')
        
# Plot detections
plt.scatter(det.geometry.x,det.geometry.y,s=1, label='Change detected')
plt.legend()

# Display detected and damages on interactive map
# Initialise map
m3 = wf.interactive.MapApp()
m3.center, m3.zoom = (lat, lon), zoom

# Plot background image and overlay damages
after = globals()['mos_2'+str(visual)].visualize('After', map=m3)  
geo_data = GeoData(geo_dataframe = damage, style={'color': 'red', 'radius':2, 'fillColor': 'red', 'opacity':0.2, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.2},
                    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                    point_style={'radius': 3, 'color': 'red', 'fillOpacity': 0.2, 'fillColor': 'red', 'weight': 3},
                    name = 'Damages')
m3.add_layer(geo_data)

# Plot detections
try: detection = detections.mask(detections==0).mask(omitMask==1).visualize('Detected Change', checkerboard=False, colormap='plasma', map=m3)
except: detection = detections.mask(detections==0).visualize('Detected Change', checkerboard=False, colormap='plasma', map=m3)
detection.opacity, after.opacity = 0.7, 0.6

# Legend
m3.add_control(LegendControl({"Detected Change":"#FFFF00", "Recorded Damage":"#FF0000"})) 

m3

________
<a id='quantify'></a>
## 4 - Quantify Accuracy
After qualitatively assessing the ratio method's performance in the previous section, we will now quantify the accuracy of the method for detecting building damage. 

For this we first define a building footprint around the recorded damage point location. We then determine the accuracy with which detected changed pixels correspond to these building footprints. The metrics are as follows:
- Precision (proportion of damage detected): $P = \frac{True Positives}{True Positives + False Positives}$
- Recall (proportion of detections corresponding to damage): $R = \frac{True Positives}{True Positives + False Negatives}$
- F1 Score: $F1 = 2x\frac{P*R}{P+R}$

In [ ]:
# Create polygons around point locations of damages
if not os.path.exists(dmgFile) and damage.geometry[damage.index[0]].type is not 'Polygon': # Gets point assessment damages into geojson file
    features = []
    for i in tqdm(damage.index):
        poly = Polygon([[damage.geometry.x[i], damage.geometry.y[i]], [damage.geometry.x[i]+area, damage.geometry.y[i]], [damage.geometry.x[i]+area, damage.geometry.y[i]+area], [damage.geometry.x[i], damage.geometry.y[i]+area], [damage.geometry.x[i], damage.geometry.y[i]]])
        features.append(geojson.Feature(properties={"Damage": damage.grading[i]}, geometry=poly))

    fc = geojson.FeatureCollection(features)
    with open(dmgFile, 'w') as f: geojson.dump(fc, f)
    
elif not os.path.exists(dmgFile):  # Puts polygon assessments into geojson file
    with open(dmgFile, 'w') as f: geojson.dump(damage, f)

In [ ]:
# Load building damages
dmg = gpd.read_file(dmgFile)
print('Changed pixels:',len(det), '\nDamaged buildings:',len(dmg))

# Initialise accuracy and recall vectors
acc, rec = np.zeros([max(dmg.index)+1,1]), np.zeros([max(det.index)+1,1]) # Initialise accuracy, recall arrays

# Loop through pixels to determine recall (if pixel corresponds to damaged building)
for i in tqdm(det.index):
    # Loop through building to determine accuracy (damaged building has been detected)
    for j in dmg.index:
        if det.geometry[i].within(dmg.geometry[j]):
            rec[i,0], acc[j,0] = True, True

# Calculate metrics from vector outputs
a = sum(acc)/len(dmg)
r = sum(rec)/len(det)
f1 = 2*(a*r)/(a+r)
print('Precision:',a[0],'\nRecall:',r[0],'\nF1 score:',f1[0])

In [ ]:
## Plot success of change detection in matplotlib and save figure
# Damage detected true/false
dmg['found'] = pd.Series(acc[:,0], index=dmg.index)
plt = dmg.plot(figsize=(12,8), column='found',legend=True,cmap='RdYlGn',alpha = 0.7)

# False detection points
points = np.vstack([rec[i] for i in det.index])
x1, y1 = np.array(det.geometry.x)*(1-points).transpose(), np.array(det.geometry.y)*(1-points).transpose()
x1, y1 = x1[x1 != 0], y1[y1 != 0]
plt.scatter(x1,y1,s=0.05,color='b', label='False detections')

# Set titles and save
plt.set_title('Threshold:'+str(threshold)+', Area:'+str(area)+', Kernel:'+str(kSize)+' - Acc:'+str(a[0])[:6]+', Re:'+str(r[0])[:6])
plt.legend()
plt.figure.savefig('results/'+location+'_t'+str(threshold)[2:]+'a'+str(area)[2:]+'g'+str(len(grades))+str(bands)+'.png')

## Display on interactive map
# Initialise map
m4 = wf.interactive.MapApp()
m4.center, m4.zoom = (lat, lon), zoom

# Plot background imagery as image 2 using function from map 1
getImage(1,visual,0.7,m4)

# Add layers for building polygons whether red for not found, green for found
not_found = GeoData(geo_dataframe = dmg.loc[dmg['found']==0], style={'color': 'red', 'radius':2, 'fillColor': 'red', 'opacity':0.7, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.7},
                    hover_style={'fillColor': 'red' , 'fillOpacity': 0.5},
                    name = 'Damages')
found = GeoData(geo_dataframe = dmg.loc[dmg['found']==1], style={'color': 'green', 'radius':2, 'fillColor': 'green', 'opacity':0.7, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.7},
                    hover_style={'fillColor': 'green' , 'fillOpacity': 0.5},
                    name = 'Damages')
m4.add_layer(not_found)
m4.add_layer(found)

# Plot pixels where change has been detected
try: detection = detections.mask(detections==0).mask(omitMask==1).visualize('Detected Change', checkerboard=False, colormap='plasma', map=m4)
except: detection = detections.mask(detections==0).visualize('Detected Change', checkerboard=False, colormap='plasma', map=m4)
detection.opacity = 0.7

# Legend
m4.add_control(LegendControl({"Detected Change":"#FFFF00", "Damage Identified":"#008000", "Damage Not Identified":"#FF0000"})) 

m4

# --------------- END ------------------------